# Problema de detección de anomalías

## Identificación de transacciones fraudulentas en tarjetas

### Descripción del problema:

Uno de los problemas en la financiación al consumo es el fraude en tarjetas, dinero que es cargado a los clientes por productos que ellos no han adquirido. Esto es un problema importante para el sector financiero ya que, además de las pérdidas monetarias, disminuye la fidelización de los clientes. Hay que recordar que el sistema monetario actual se basa en la confianza, y el dinero es lo primero que escapa cuando no hay confianza en las entidades.

Los datos de este problema provienen de la siguiente dirección de Kaggle:

**Url:** https://www.kaggle.com/mlg-ulb/creditcardfraud

### Descripción del dataset:

Cuenta con un total de **30 variables predictoras X** y una **variable continua a predecir Y**.

El número total de muestras es de 284.807 transacciones.

El dataset contiene transacciones de tarjetas realizadas en septiembre de 2013 por entidades europeas.
Estas transacciones ocurrieron en dos días, produciéndose un total de 492 transacciones fraudulentas de un total de 284.807.
El dataset está altamente desbalanceado, la clase positiva (fraudes) sólo es un 0.172 % del total de transacciones.

**Información de las variables:**

**Variable dependiente Y:**

La variable 'Class' es la respuesta, indica si esa transacción es fraudulenta (1) o no es fraudulenta (0).

**Variables independientes X:**

Todas las variables en este dataset son numericas. Dos de ellas son 'Time' y 'Amount', las cuales son variables originales. El resto, son resultado de una transformación PCA, siendo V1-V28 las componentes principales. Por motivos de confidencialidad no se proporcionan las variables originales ni más información sobre los datos. 

* 'Time': segundos entre cada transacción y la primera transacción del dataset
* 'Amount': unidades monetarias de la transacción
* Variables 'V1-V28': componentes principales de una transformación de variables utilizando PCA

### Planteamiento del problema

#### Contexto :

En un escenario del mundo real, los algoritmos no supervisados se utilizan en estos casos para filtrar del total de transacciones cuáles son anómalas. A veces se utilizan reglas de negocio para hacer ese filtrado, pero muchas veces esas reglas son muy difíciles de formular. Ya que no es viable analizar todas las transacciones, un modelo no supervisado que filtre comportamientos extraños es muy útil para realizar una primera criba. Para ello se hallan características de cada transacción (X) y con detectores de anomalías se estima cuáles son anómalas y cuáles no. 

Posteriormente, se auditan todas esas transacciones anómalas y se obtienen las etiquetas reales (Y). Se halla qué transacciones son fraudulentas (Y=1) y cuales no (Y=0).

Llegado un punto, la entidad financiera tiene datos etiquetados (X,Y) que puede utilizar para realizar un clasificador de aprendizaje supervisado.  

#### Planteamiento de este ejemplo:

En este caso tenemos las características de cada transacción (X) y la etiqueta indicando si son fraude o no (Y). Podríamos realizar directamente un clasificador de aprendizaje supervisado, pero el objetivo es que tengáis una aproximación a cómo de útil puede ser un modelo de detección de anomalías en pasos previos.

Voy a suponer que no existe una etiqueta (Y), voy a realizar un modelo de detección de anomalías sólamente con las características (X). Hallaré qué transacciones obtengo como anómalas y luego compararé con las etiquetas reales (Y). De este modo podremos ver cómo un detector de anomalías podría servir en ese primer filtrado.

# Carga de librerías:

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
new_style = {'grid': False}
plt.rc('axes', **new_style)

import seaborn as sns

from sklearn.neighbors import LocalOutlierFactor
import sklearn.metrics as metrics

import scikitplot as skplt # ! pip install scikit-plot

# Definición de funciones: 

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def repre_matriz_confusion(matriz):
    df_matriz_confusion = pd.DataFrame(matriz,
                     ['True Normal','True Fraud'],
                     ['Pred Normal','Pred Fraud'])
    plt.figure(figsize = (8,4))
    sns.set(font_scale=1.4)
    plt.title(u'Matriz de confusión')
    _ = sns.heatmap(df_matriz_confusion, annot=True, annot_kws={"size": 16}, fmt='g')
    
def reporting_modelo(y_reales, y_clase):
    matriz_confusion = metrics.confusion_matrix(y_reales, y_clase)
    roc_auc = metrics.roc_auc_score(y_reales, y_clase)
    metrica_f1 = metrics.f1_score(y_reales, y_clase)
    print(u'La AUC de la ROC es de: {}'.format(round(roc_auc,2)))
    print(u'La F1 es de: {}'.format(round(metrica_f1,2)))
    print("\nAccuracy\t{}".format(round(metrics.accuracy_score(y_reales, y_clase),3)))  
    print("Sensitividad\t{}".format(round(metrics.recall_score(y_reales, y_clase),3)))
    print(u"Precisión\t{}".format(round(metrics.precision_score(y_reales, y_clase),3)))   
    repre_matriz_confusion(matriz_confusion)
    
def repres_doble_hist(y_prob_pos, y_prob_neg):
    
    fig = plt.figure(figsize=(20,10))
    ax = sns.distplot(y_prob_pos,norm_hist=True, bins=30, hist=False,
    label='', kde_kws={"color": "r", "lw": 5})  
    ax2 = ax.twinx()
    sns.distplot(y_prob_neg,norm_hist=True ,ax=ax2, bins=30, hist=False,
    label='', kde_kws={"color": "g", "lw": 2}) 
    sns.set_style("whitegrid", {'axes.grid' : False})
    ax.figure.legend(['Clase fraudulenta', 'Clase no fraudulenta'])
    new_style = {'grid': False}
    plt.rc('axes', **new_style)
    plt.title('Representación de las probabilidades asignadas a ambas clases')
    plt.show()

# Lectura de datos:

In [ ]:
XY = pd.read_csv('creditcard.csv')

In [ ]:
print(u'- El número de filas en el dataset es: {}'.format(XY.shape[0]))
print(u'- El número de columnas en el dataset es: {}'.format(XY.shape[1]))
print(u'- Los nombres de las variables son: {}'.format(list(XY.columns)))
XY[:2]

In [ ]:
XY['Class'].value_counts()

In [ ]:
XY['Class'].value_counts().plot(kind='pie', figsize=(7,7))
_ = plt.title('Distribución de transacciones', fontsize=20)

Como vemos, las clases están muy desbalanceadas ya que apenas hay casos fraudulentos. 

# Division en features X + target Y 

In [ ]:
X = XY.drop('Class', axis=1)
Y = XY['Class']

# Ajuste del modelo de detección de anomalías a X

Voy a utilizar el método Local Outlier Factor. Este método sólo se fija en los vecinos locales de cada punto por lo que vamos a ajustar el modelo a todos los datos X. 

Un parámetro que se puede ir cambiando es el **número de vecinos**. A menor número más se ajusta el modelo.

In [ ]:
clf=LocalOutlierFactor(n_neighbors=10, 
                        algorithm='auto', 
                        leaf_size=30,
                        metric='minkowski', 
                        p=2, 
                        metric_params=None, 
                        n_jobs=-1,
                        novelty=False)

In [ ]:
%%time
clf.fit(X)

# Detección de anomalías

El modelo nos proporciona lo que se denominan factores de anomalías negativos. Cuanto más alto es este valor, más normal es el punto. Es decir, si queremos quedarnos con el 2 % de puntos más anómalos, debemos quedarnos con el 2 % de valores más bajo. 

In [ ]:
factores_lof = clf.negative_outlier_factor_
factores_lof

Ponemos el umbral en un 2 % para tener margen.

In [ ]:
Y_pred_clase = factores_lof.copy()
Y_pred_clase[factores_lof>=np.percentile(factores_lof,2.)] = 0
Y_pred_clase[factores_lof<np.percentile(factores_lof,2.)] = 1

In [ ]:
 reporting_modelo(Y, Y_pred_clase) 

Como vemos, con un dos por ciento de la muestra detectada como fraude, detectamos más del 50 % de las transacciones fraudulentas.

## Representación de las probabilidades:

Normalizo los factores al rango (0,1) para obtener una estimación de probabilidades:

In [ ]:
Y_probs = NormalizeData(factores_lof)
Y_pred_prob_pos = NormalizeData(factores_lof)[np.where(Y == 1)]
Y_pred_prob_neg = NormalizeData(factores_lof)[np.where(Y == 0)]

In [ ]:
repres_doble_hist(Y_pred_prob_pos, Y_pred_prob_neg)

El modelo asigna a la clase fraudulenta valores centrados en el 1 (anómalos). Por otro lado, la clase no fraudulenta tiene muchos valores en 0s y en 1s. 